In [402]:
import pandas as pd
import numpy as np

import datetime 
from datetime import timedelta

In [419]:
filename = 'datasets/JC-201907-citibike-tripdata.csv'

In [420]:
df = pd.read_csv(filename)
df["tripduration"]= df["tripduration"].astype(int) 
df["start station id"]= df["start station id"].astype(int) 
df["end station id"]= df["end station id"].astype(int) 
df["birth year"]= df["birth year"].astype(int) 
df["gender"]= df["gender"].astype(int) 
df["starttime"]= pd.to_datetime(df["starttime"], format="%Y-%m-%d %H:%M:%S.%f")
df["stoptime"] = pd.to_datetime(df["stoptime"], format="%Y-%m-%d %H:%M:%S.%f")


df.drop(['tripduration', 'usertype', 'start station latitude', 'start station longitude', 'end station longitude', 'end station latitude', 'birth year', 'gender', 'bikeid'], axis=1, inplace=True)

In [421]:
df.head()

,starttime,stoptime,start station id,start station name,end station id,end station name
0,2019-07-01 00:00:36.615,2019-07-01 00:30:30.594,3186,Grove St PATH,3198,Heights Elevator
1,2019-07-01 00:02:54.615,2019-07-01 00:10:09.037,3206,Hilltop,3280,Astor Place
2,2019-07-01 00:03:27.567,2019-07-01 00:07:38.608,3186,Grove St PATH,3205,JC Medical Center
3,2019-07-01 00:03:56.169,2019-07-01 00:14:32.418,3206,Hilltop,3281,Leonard Gordon Park
4,2019-07-01 00:04:16.206,2019-07-01 00:14:40.050,3206,Hilltop,3281,Leonard Gordon Park


In [422]:
# start time id

sorted_by_time = df.sort_values(by=['starttime'])

sorted_by_time.insert(1, "diff_start", sorted_by_time['starttime'].diff(), True)
sorted_by_time.insert(1, "id_start", None, True) 


group_id = 0
for idx, r in sorted_by_time.iterrows():
    if r.diff_start > datetime.timedelta(minutes=5):
        group_id+=1
    sorted_by_time['id_start'][idx] = group_id

<ipython-input-422-7cc42bc5c4b9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_by_time['id_start'][idx] = group_id


In [423]:
# end time id

sorted_by_time = sorted_by_time.sort_values(by=['stoptime'])

sorted_by_time.insert(1, "diff_end", sorted_by_time['stoptime'].diff(), True) 
sorted_by_time.insert(1, "id_end", None, True) 


group_id = 0
for idx, r in sorted_by_time.iterrows():
    if r.diff_end > datetime.timedelta(minutes=5):
        group_id+=1
    sorted_by_time['id_end'][idx] = group_id

<ipython-input-423-07948047a047>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_by_time['id_end'][idx] = group_id


In [424]:
sorted_by_time.insert(1, 'group_id', sorted_by_time.groupby(['start station name', 'end station name', 'id_start', 'id_end']).ngroup(), True)

In [425]:
#dropping all groups with less than 2 people
groups = sorted_by_time[sorted_by_time.duplicated(subset=['group_id'], keep=False)]

In [426]:
#count of all group rides

groups.group_id.nunique()

6883